# Visualization of Transformer Prediction on a map

In [1]:
import torch
import torch.nn as nn
from tptrans import TPTrans
from dataloader import load_parquet, preprocess_data, SlidingWindowDataset
from plotting import plot_testresult_sample
from torch.utils.data import DataLoader

In [2]:
#in your case it will be
#dataset = "dataset"
dataset = "dataset"

In [3]:
df = load_parquet(dataset, k=5)
df = preprocess_data(df)

dataset = SlidingWindowDataset(
    df,
    max_diff_per_sequence_minutes=15,
    window_size_minutes=120,
    pred_size_minutes=30,
    stride=15,
)
test_loader = DataLoader(dataset, batch_size=1, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TPTrans(pred_len=dataset[0][1].shape[0]).to(device)
#
model.load_state_dict(torch.load("checkpoints/tptrans_delta_lin_newNewParams_375.pth", map_location=torch.device(device)))
#model.load_state_dict(torch.load("/zhome/63/7/219953/DeepLearning-1/checkpoints/tpinformer_delta_lin_newNewParams_375.pth", map_location=torch.device(device)))
model.eval()

criterion = nn.MSELoss()
test_loss = 0.0
data = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        tgt_start = x[:, -1:, :2]  # (batch, 1, 2) lat/lon
        pred = model(x, tgt_start)
        data.append((x,y,pred))
        loss = criterion(pred, y)
        test_loss += loss.item() * x.size(0)
test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.6f}", flush=True)

plot_testresult_sample(data)

/zhome/63/7/219953/DeepLearning-1/dataloader.py:167: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df.set_index("Timestamp")
/zhome/63/7/219953/DeepLearning-1/dataloader.py:167: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.set_index("Timestamp")
/zhome/63/7/219953/.local/lib/python3.9/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


RuntimeError: Error(s) in loading state_dict for TPTrans:
	Missing key(s) in state_dict: "input_proj.weight", "input_proj.bias", "encoder.layers.4.self_attn.in_proj_weight", "encoder.layers.4.self_attn.in_proj_bias", "encoder.layers.4.self_attn.out_proj.weight", "encoder.layers.4.self_attn.out_proj.bias", "encoder.layers.4.linear1.weight", "encoder.layers.4.linear1.bias", "encoder.layers.4.linear2.weight", "encoder.layers.4.linear2.bias", "encoder.layers.4.norm1.weight", "encoder.layers.4.norm1.bias", "encoder.layers.4.norm2.weight", "encoder.layers.4.norm2.bias", "encoder.layers.5.self_attn.in_proj_weight", "encoder.layers.5.self_attn.in_proj_bias", "encoder.layers.5.self_attn.out_proj.weight", "encoder.layers.5.self_attn.out_proj.bias", "encoder.layers.5.linear1.weight", "encoder.layers.5.linear1.bias", "encoder.layers.5.linear2.weight", "encoder.layers.5.linear2.bias", "encoder.layers.5.norm1.weight", "encoder.layers.5.norm1.bias", "encoder.layers.5.norm2.weight", "encoder.layers.5.